### Run this to import rental flats and houses from Luzern with a radius of 5km.

* This search condition is defined by this URL parameter: **l=2253&r=5&s=1&t=1**
* To search for Zurich, change **l=2253** to **l=4147** (for other cities, check the webpage www.immoscout24.ch)
* For incremental import (e.g. additionally add new rental properties from the last 30 days), add: **&pa=30**

In [1]:
import pandas as pd
import pytz
import time
import datetime
import urllib
import json
from shutil import copyfile

def requestApi(url):
    response = urllib.request.urlopen(url)
    document = json.loads(response.read())
    return document

def writeJsonFile(pathFileName, data):
    with open(pathFileName, 'w') as outfile:
        json.dump(data, outfile)

def readJsonFile(pathFileName):
    with open(pathFileName) as json_file:
        return json.load(json_file)

today_st = str(datetime.date.today())
list_file = './data/processed_list_df.csv'
document_file = './data/rent_documents.json'
    
api_url = 'https://rest-api.immoscout24.ch/v4/de/properties'
# add &pa=30 to retrieve newest rental from the last 30 days
# param l is the id for the city (i.e. 2253 is Luzern, 4147 for Zürich etc.)
url_listing =  api_url + '?l=2253&r=5&s=1&t=1' 

response_listing = urllib.request.urlopen(url_listing)
document_list_resp = json.loads(response_listing.read())

document_json = readJsonFile(document_file)
processed_df = pd.read_csv(list_file)
processed_id_list = processed_df['id'].tolist()

print('Before import job run, count: ' + str(len(processed_df)))

print('Importing, please wait ...')

for item in document_list_resp['allProperties']:
    if item['id'] in processed_id_list:
        continue
    time.sleep(1)
    url = api_url + '/' + str(item['id'])
    document_json.append(requestApi(url))
    processed_df = processed_df.append({'id': int(item['id']), 
                                        'lat': round(item['latitude'], 5), 
                                        'long': round(item['longitude'], 5), 
                                        'geoAccuracy': round(item['geoAccuracy'], 2)
                                       }, ignore_index=True)

processed_df = processed_df.sort_values(by=['id'])
processed_df['id'] = processed_df['id'].astype(int)

copyfile(list_file, list_file + '_copy_' + today_st)
copyfile(document_file, document_file + '_copy_' + today_st)

processed_df.to_csv(list_file, index=False)
writeJsonFile(document_file, document_json)

print("Finish importing from API, count: " + str(len(processed_df)))


Before import job run, count: 51
Importing, please wait ...
Finish importing from API, count: 711
